# Notebook description
<!---define operatorname var-->
$\DeclareMathOperator{\Var}{Var}$
<!---define operatorname var-->

This notebook looks at the signal to noise ratio (SNR) of a synapse with input spike train with frequency $\lambda$ and interspike intervals (ISIs) following a Gamma distribition.

The pdf of the gamma distribution is given by
$$
f_X(x) = \frac{\lambda^k}{\Gamma(k)}x^{k-1}e^{-\lambda x}
$$

We are particularly interested in cases when $k$ is a positive integer, in which case

$$
f_X(x) = \frac{\lambda^k}{(k-1)!}x^{k-1}e^{-\lambda x}
$$

which is the pdf of the Erlang distribution.

# Summary of results

<div style="display:block; border: 1px solid black; padding: 15px;background-color: #EEEEFF;margin: 10px">
\begin{align*}
E[X] &= \\
\Var(X) &=  \\
SNR &=  \\
\end{align*}
</div>

# Derivations

We [know](http://nbviewer.ipython.org/github/fragapanagos/notebooks/blob/master/theory/campbell_theorem.ipynb#Filtered-Renewal-processes) that first-order, low-passed filtered renewal processes have moments
\begin{align*}
E[X] &= \frac{1}{\tau}\sum_{i=1}^\infty E\left[e^{-T/\tau}\right]^i \\
E[X^2] &= \frac{1}{\tau^2}\sum_{i=1}^{\infty}E\left[e^{-2T/\tau}\right]^i\left(1 +
    2\sum_{j=1}^{\infty}E\left[e^{-T/\tau}\right]^{j}\right) \\
\end{align*}

Therefore, our first steps will be to compute $E\left[e^{-2T/\tau}\right]$ and $E\left[e^{-T/\tau}\right]$, or rather $E\left[e^{-cT/\tau}\right]$.

\begin{align*}
E[e^{-cT/\tau}] &= \int_{-\infty}^{\infty}e^{-ct/\tau}f_T(t)dt \\
 &= \int_{0}^{\infty}e^{-ct/\tau}\frac{\lambda^k}{\Gamma(k)}t^{k-1}e^{-\lambda t}dt \\
 &= \frac{\lambda^k}{\Gamma(k)}\int_{0}^{\infty}t^{k-1}e^{-ct/\tau}e^{-\lambda t}dt \\
 &= \frac{\lambda^k}{\Gamma(k)}\int_{0}^{\infty}t^{k-1}e^{-t(c/\tau-\lambda)}dt \\
 &= \frac{\lambda^k}{\Gamma(k)}\int_{0}^{\infty}\left(\frac{\tau}{c-\lambda\tau}z\right)^{k-1}
     e^{-z}\frac{\tau}{c-\lambda\tau}dz & \text{(see below) where } z = t(c/\tau-\lambda) \\
 &= \frac{\lambda^k}{\Gamma(k)}\int_{0}^{\infty}\left(\frac{\tau}{c-\lambda\tau}\right)^kz^{k-1}e^{-z}dz \\
 &= \frac{\lambda^k}{\Gamma(k)}\left(\frac{\tau}{c-\lambda\tau}\right)^k\int_{0}^{\infty}z^{k-1}e^{-z}dz \\
 &= \frac{\lambda^k}{\Gamma(k)}\left(\frac{\tau}{c-\lambda\tau}\right)^k\Gamma(k) \\
 &= \left(\frac{\lambda\tau}{c-\lambda\tau}\right)^k \\
\end{align*}

Here, we expand on our substitution of $z$ for $t$.

\begin{align*}
z &= t(c/\tau-\lambda) \\
 &= t\frac{c-\lambda\tau}{\tau} \\
\frac{\tau}{c-\lambda\tau}z &= t \\
\frac{\tau}{c-\lambda\tau}dz &= dt \\
\end{align*}

Note that this substitution requires $c>\lambda\tau$ for the limits to be consistent.

## Synapse mean

\begin{align*}
E[X] &= \frac{1}{\tau}\sum_{i=1}^\infty E\left[e^{-T/\tau}\right]^i \\
 &= \frac{1}{\tau}\sum_{i=1}^\infty\left(\frac{\lambda\tau}{1-\lambda\tau}\right)^{ki} \\
 &= \frac{1}{\tau}\frac{\left(\frac{\lambda\tau}{1-\lambda\tau}\right)^k}
     {1-\left(\frac{\lambda\tau}{1-\lambda\tau}\right)^k} &
     \text{(see below) assuming } \left|\frac{\lambda\tau}{1-\lambda\tau}\right|<1 \\
 &= \frac{1}{\tau}\frac{(\lambda\tau)^k}{(1-\lambda\tau)^k-(\lambda\tau)^k}
\end{align*}

Here, we look at the implications of our assumption required for the summation to converge

$$
\left|\frac{\lambda\tau}{1-\lambda\tau}\right|<1
$$

becomes

\begin{align*}
\frac{\lambda\tau}{1-\lambda\tau} &< 1 \\
\lambda\tau &< 1-\lambda\tau \\
\lambda\tau &< \frac{1}{2} \\
\end{align*}

or

\begin{align*}
-\frac{\lambda\tau}{1-\lambda\tau} &< 1 \\
\frac{\lambda\tau}{1-\lambda\tau} &\ge -1 \\
\lambda\tau &\ge -1+\lambda\tau \\
0 &\ge -1 \\
\end{align*}

which is trivally true. Therefore, we only require that $\lambda\tau<\frac{1}{2}$ (i.e. less than 1 spike per $\tau$).


## Syanpse variance

\begin{align*}
E[X^2] &= \frac{1}{\tau^2}\sum_{i=1}^{\infty}E\left[e^{-2T/\tau}\right]^i\left(1 +
    2\sum_{j=1}^{\infty}E\left[e^{-T/\tau}\right]^{j}\right) \\
 &= \frac{1}{\tau^2}\sum_{i=1}^{\infty}\left(\frac{\lambda\tau}{2-\lambda\tau}\right)^{ki}\left(1 +
    2\sum_{j=1}^{\infty}\left(\frac{\lambda\tau}{1-\lambda\tau}\right)^{kj}\right) \\
 &= \frac{1}{\tau^2}\frac{(\lambda\tau)^k}{(2-\lambda\tau)^k-(\lambda\tau)^k}\left(1 +
    2\frac{(\lambda\tau)^k}{(1-\lambda\tau)^k-(\lambda\tau)^k}\right) \\
 &= \frac{1}{\tau^2}\left(\frac{(\lambda\tau)^k}{(2-\lambda\tau)^k-(\lambda\tau)^k} +
    2\frac{(\lambda\tau)^k}{(2-\lambda\tau)^k-(\lambda\tau)^k}
    \frac{(\lambda\tau)^k}{(1-\lambda\tau)^k-(\lambda\tau)^k}\right) \\
 &= \frac{1}{\tau^2}\left(\frac{(\lambda\tau)^k\left[(1-\lambda\tau)^k-(\lambda\tau)^k\right]+2(\lambda\tau)^{2k}}
    {\left[(2-\lambda\tau)^k-(\lambda\tau)^k\right]\left[(1-\lambda\tau)^k-(\lambda\tau)^k\right]}\right) \\
 &= \frac{1}{\tau^2}\left(\frac{(\lambda\tau)^k\left[(1-\lambda\tau)^k+(\lambda\tau)^k\right]}
    {\left[(2-\lambda\tau)^k-(\lambda\tau)^k\right]\left[(1-\lambda\tau)^k-(\lambda\tau)^k\right]}\right) \\
\end{align*}

\begin{align*}
\Var(X) &= E[X^2] - E[X]^2 \\
 &= \frac{1}{\tau^2}\left(\frac{(\lambda\tau)^k\left[(1-\lambda\tau)^k+(\lambda\tau)^k\right]}
    {\left[(2-\lambda\tau)^k-(\lambda\tau)^k\right]\left[(1-\lambda\tau)^k-(\lambda\tau)^k\right]}\right)
    - \left(\frac{1}{\tau}\frac{(\lambda\tau)^k}{(1-\lambda\tau)^k-(\lambda\tau)^k}\right)^2
\end{align*}

This is pretty ugly...